<a href="https://colab.research.google.com/github/AHartNtkn/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS7_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-08-19 19:00:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2019-08-19 19:00:45 (624 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [10]:
#import
df = pd.read_csv('house-votes-84.data',names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

#clean up the values
for c in df.columns:
  if c != 'party':
    df[c] = df[c].apply(lambda x: 0 if x == 'n' else (1 if x == 'y' else np.NaN))

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
repData = df[df['party']=='republican'].drop('party', axis=1)
demData = df[df['party']=='democrat'].drop('party', axis=1)

In [16]:
# Let's perform a t-test on everything.
for c in repData.columns:
  print(c+':', ttest_ind(repData[c], demData[c], nan_policy='omit'))

handicapped-infants: Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
water-project: Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
budget: Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
physician-fee-freeze: Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
el-salvador-aid: Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
religious-groups: Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
anti-satellite-ban: Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
aid-to-contras: Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
mx-missile: Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
immigration: Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
synfuels: Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064

As can be seen from the results, the republicans favor the religious-groups bill at p<.01, the democrats favor the budget bill at p<.01, and the difference on the water-project is insignificant.

I'm not really sure what it expects for function refactoring, but here's a function that performs a hypothesis test on a provided bill.

In [29]:
def billTest(bill):
  res = ttest_ind(repData[bill], demData[bill], nan_policy='omit')
  
  print(bill+' is supported by ' + ('republicans' if res.statistic > 0 else 'democrats') + 
        ' at p=' + str(res.pvalue)+'.')
  
for c in repData.columns:
  billTest(c)

handicapped-infants is supported by democrats at p=1.613440327937243e-18.
water-project is supported by republicans at p=0.9291556823993485.
budget is supported by democrats at p=2.0703402795404463e-77.
physician-fee-freeze is supported by republicans at p=1.994262314074344e-177.
el-salvador-aid is supported by republicans at p=5.600520111729011e-68.
religious-groups is supported by republicans at p=2.3936722520597287e-20.
anti-satellite-ban is supported by democrats at p=8.521033017443867e-31.
aid-to-contras is supported by democrats at p=2.82471841372357e-54.
mx-missile is supported by democrats at p=5.03079265310811e-47.
immigration is supported by republicans at p=0.08330248490425066.
synfuels is supported by democrats at p=1.5759322301054064e-15.
education is supported by republicans at p=1.8834203990450192e-64.
right-to-sue is supported by republicans at p=1.2278581709672758e-34.
crime is supported by republicans at p=9.952342705606092e-47.
duty-free is supported by democrats at 

I also used a statistical test in my project. I tested whether certain trends had statistical significance; specifically Position of Negative Word With Respect to Subject, Object, and Verb seemed like it might have an interesting and statistically significant result. It did not.